In [ ]:
from get_data import *
from simulation import *
from optimization import *
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import tqdm
import seaborn as sns
import os
import glob
import warnings
warnings.filterwarnings('ignore')

# All Optimization Results

In [ ]:
res_dict = {}  # Target dictionary to store results, don't update this dictionary for testing different objective function params
files_df = get_path_df()
gb = files_df.groupby(['stock_code', 'side', 'ts', 'tm'])
for k, df in tqdm.tqdm(gb, desc='GettingSimData', ncols=200, total=len(gb.groups)):
    
    # Get data 
    if k not in res_dict.keys():
        res_dict[k] = get_sim_res(path_ls=df['path'].unique().tolist())
    
    # Calculate score
    res_score = obj(res_dict[k]['pnl'], res_dict[k]['duration'], **{'lmda': 1, 't_func': lambda x: np.sqrt(x / 1000.)})
    res_score = res_score[np.logical_and(~np.isnan(res_score), ~np.isinf(res_score))]
    res_dict[k].update({
        'scores': res_score, 
        'count_k': len(res_score) / 1000, 
        'score_mean': np.nanmean(res_score), 
        'score_std': np.nanstd(res_score)
    })

In [ ]:
plot_heatmap(stock_code_ls=['0050', '2330'], side_ls=['bid', 'ask'], plot_attri_ls=['score_mean', 'score_std', 'count_k'], res_dict=res_dict)